# Exemple de génération de trace GPX à partir des points de la trace RTK exportée en format GeoJSON depuis l'application SW MAPS

## Données d'entrée : fichier geojson des points de la trace RTK collectés lors de la sortie de terrain de Lorgues :

### 1) Installation de la bibliothèque python gpxpy pour la création de la trace GPX :

In [18]:
!pip install gpxpy

   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2) Installation de la bibliothèque python geojson pour le chargement du fichier geojson d'entrée :

In [20]:
!pip install geojson


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 3) Extraction des points RTK avec leurs horodatages en liste et les convertir en DataFrame :

In [1]:
import geojson
import pandas as pd

# Charger le fichier GeoJSON
with open(r"..\terrain_data\raw\Lorgues\Trace Lorgues points.geojson", 'r') as f:
    data = geojson.load(f)

# Extraire les points et leurs horodatages
points = []
for feature in data['features']:
    if feature['geometry']['type'] == 'Point':
        coords = feature['geometry']['coordinates']
        timestamp = feature['properties'].get('time')
        points.append({
            'latitude': coords[1],
            'longitude': coords[0],
            'elevation': coords[2] if len(coords) > 2 else None,
            'time': timestamp
        })

# Convertir en DataFrame
df = pd.DataFrame(points)
df

,latitude,longitude,elevation,time
0,43.492107,6.363533,246.386,01-07-2024 12:33:43.600 GMT+02:00
1,43.492161,6.363544,246.539,01-07-2024 12:33:51.600 GMT+02:00
2,43.492241,6.363510,245.188,01-07-2024 12:33:59.600 GMT+02:00
3,43.492298,6.363446,242.423,01-07-2024 12:34:06.800 GMT+02:00
4,43.492456,6.363327,241.550,01-07-2024 12:34:22.000 GMT+02:00
...,...,...,...,...
116,43.492508,6.363121,246.353,01-07-2024 12:51:40.400 GMT+02:00
117,43.492431,6.363172,246.116,01-07-2024 12:51:46.800 GMT+02:00
118,43.492309,6.363228,246.705,01-07-2024 12:51:56.400 GMT+02:00
119,43.492198,6.363308,247.928,01-07-2024 12:52:05.200 GMT+02:00


### 4) Installation de la bibliothèque python dateutil pour le formatage de la date à partir d'une chaine de caractères :

In [40]:
pip install python-dateutil

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 5) Création de la trace GPX en ajoutant les points horodatées au segment de la trace :

In [2]:
import gpxpy
import gpxpy.gpx
from dateutil import parser

# Créer un objet GPX
gpx = gpxpy.gpx.GPX()

# Créer une nouvelle trace
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Ajouter un segment à la trace
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

# Ajouter les points au segment
for idx, row in df.iterrows():
# Convertir l'horodatage en format datetime
    time = parser.parse(row['time']) if row['time'] else None
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
        latitude=row['latitude'],
        longitude=row['longitude'],
        elevation=row['elevation'],
        time=time
    ))

### 6) Exportation de la trace GPX :

In [3]:
# Écrire le fichier GPX
with open(r"..\terrain_data\processed\Lorgues\Trace Lorgues.gpx", 'w') as f:
    f.write(gpx.to_xml())